In [ ]:
# Install required packages

!pip install tensorflow==2.18.0
!pip install keras==3.7.0
!pip install torch==2.5.1
!pip install torchvision==0.20.1

!pip install numpy==2.0.2
!pip install scipy==1.14.1
!pip install pandas==2.2.3

!pip install scikit-learn==1.5.2

!pip install matplotlib==3.9.2

!pip install joblib==1.4.2
!pip install python-dateutil==2.9.0.post0

!pip install sympy==1.13.1
!pip install opt-einsum==3.4.0

!pip install tensorboard==2.18.0
!pip install protobuf==5.29.0
!pip install threadpoolctl==3.5.0
!pip install packaging==24.2


#1. Import Necessary Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

* numpy: For numerical operations, including dataset generation and calculations.
* matplotlib.pyplot: To visualize results as plots.
* PolynomialFeatures: Expands input features into polynomial terms to simulate varying model complexity.
* Ridge: Linear regression with optional L2 regularization to stabilize the solutions.
* mean_squared_error: Metric for calculating errors between predicted and actual values.
* train_test_split: Splits data into training and testing sets for evaluating generalization performance.


#2. Generate Synthetic Dataset

In [ ]:
# Generate synthetic dataset
np.random.seed(42)
n_samples = 30
X = np.random.uniform(-1, 1, size=(n_samples, 1))
y = np.sin(2 * np.pi * X).ravel() + 0.3 * np.random.normal(size=n_samples)

Dataset Generation:
* X: *Uniformly distributed data points between -1 and 1*.
* y: Target values created using a sine wave with added Gaussian noise (*𝜎 = 0.3*).
* **Purpose**: Simulates a regression task with moderate noise to observe overfitting and the impact of regularization.

#3. Set Up Dropout Rates and Results Storage

In [ ]:
# Dropout rates to analyze
dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4]

# Initialize results
degrees = np.arange(1, 50)
n_trials = 10
results = {rate: {'train_means': [], 'test_means': [], 'test_vars': []} for rate in dropout_rates}

* dropout_rates: Defines a range of dropout rates to test their effect on overfitting and double descent.
* degrees: Specifies the range of polynomial degrees (*model complexities*) to be tested.
* n_trials: Number of trials for each configuration to average results and calculate variance.
* results: A dictionary to store:
* **Train and Test Means**: *Average errors across trials*.
* **Test Variances**: Variance in test errors to measure stability.

#4. Define Dropout Function

In [ ]:
# Function to simulate dropout (artificially setting weights to zero)
def apply_dropout(data, rate):
    mask = np.random.binomial(1, 1 - rate, size=data.shape)
    return data * mask

apply_dropout:
* **Purpose**: Simulates dropout by randomly zeroing out elements in the data matrix based on the specified dropout rate.
* **Implimentation**: binomial(1, 1 - rate): Creates a binary mask (*1 to keep, 0 to drop*), which is element-wise multiplied with the input data.

#5. Iterate Over Dropout Rates and Degrees

In [ ]:
for dropout_rate in dropout_rates:
    train_errors_trials = []
    test_errors_trials = []

    for _ in range(n_trials):
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=np.random.randint(1000))

        train_errors = []
        test_errors = []

        for degree in degrees:
            poly = PolynomialFeatures(degree=degree)
            X_train_poly = apply_dropout(poly.fit_transform(X_train), dropout_rate)
            X_test_poly = apply_dropout(poly.transform(X_test), dropout_rate)

            model = Ridge(alpha=0.0)
            model.fit(X_train_poly, y_train)

            y_train_pred = model.predict(X_train_poly)
            y_test_pred = model.predict(X_test_poly)

            train_errors.append(mean_squared_error(y_train, y_train_pred))
            test_errors.append(mean_squared_error(y_test, y_test_pred))

        train_errors_trials.append(train_errors)
        test_errors_trials.append(test_errors)

    # Calculate means and variances
    results[dropout_rate]['train_means'] = np.mean(train_errors_trials, axis=0)
    results[dropout_rate]['test_means'] = np.mean(test_errors_trials, axis=0)
    results[dropout_rate]['test_vars'] = np.var(test_errors_trials, axis=0)

1. Outer Loop (**Dropout Rates**):

* Iterates through each specified dropout rate.
* For each rate, multiple trials are run to average the results.
2. Inner Loop (**Trials**):

* train_test_split: Randomly splits the dataset for each trial to ensure varied results.
3. Innermost Loop (**Degrees**):

* PolynomialFeatures: Expands input features to the specified degree.
* apply_dropout: Applies dropout to simulate the regularization effect.
* Ridge(alpha=0.0): Fits a regularized model with polynomial features.
4. **Error Calculations**:

* Training and testing errors are calculated for each polynomial degree.
* Results are stored for averaging and variance calculation.

#6. Visualize Results

In [ ]:
# Plot results
plt.figure(figsize=(12, 8))
for dropout_rate in dropout_rates:
    plt.plot(degrees, results[dropout_rate]['test_means'], label=f"Test Loss (Dropout={dropout_rate})", marker='o')
    plt.fill_between(degrees,
                     results[dropout_rate]['test_means'] - np.sqrt(results[dropout_rate]['test_vars']),
                     results[dropout_rate]['test_means'] + np.sqrt(results[dropout_rate]['test_vars']),
                     alpha=0.2)

plt.xlabel("Model Complexity (Polynomial Degree)")
plt.ylabel("Mean Squared Error (Log Scale)")
plt.yscale("log")
plt.title("Effect of Dropout Regularization on Double Descent")
plt.legend()
plt.grid()
plt.show()

Plot Setup:
* **X-Axis**: Polynomial degree (*model complexity*).
* **Y-Axis**: Mean squared error on a logarithmic scale for better visualization.

Data Representation:
* test_means: Average test errors plotted for each dropout rate.
* **Shaded Region**: Represents test error variance, visualized as *one standard deviation* from the mean.

Annotations:
* Title, axis labels, and legend *describe the results clearly*.